In [26]:
import sqlite3
import pandas as pd
import requests, json

#### DB 열고 store 불러온다

```python
con = sqlite3.connect("db.sqlite3")
cur = con.cursor()
df_store = pd.read_sql_query("SELECT * from api_store", con)
con.close()

stores = df_store.loc[:,['id','store_name','category','address']]
stores['len']=stores['address'].str.split(" ").str.len()
k = stores.sort_values(by='len')
k = k[k['len']<4] # 주소를 공백으로 나눴을 때 4 미만이라면, 예) 대전시 유성구 덕명동 이런식으로 정보가 부족함
# 주소가 없는 것의 개수
stores['address'].isnull().sum() # 57개


import requests,json
tmp = pd.merge(df_store,k,left_on='id', right_on='store_id',how='inner')


# 카카오 API 써서 주소가 덜 작성된거는 위도 경도로 새 주소로 업뎃해쥼
headers = {
"Authorization":"KakaoAK 66447c8b7057c042f65ac177c53ea385"
}
def getAddress(row):
    
    params = (
        ('x', row['longitude']),
        ('y', row['latitude']),
        ('input_coord', 'WGS84'),
    )
    print(row['id'])

    response = requests.get('https://dapi.kakao.com/v2/local/geo/coord2address.json', headers=headers, params=params)
    res = response.json()['meta']['total_count']
    address = None
    try:
        if response.json()['documents'][0]['road_address'] is None:
            address =response.json()['documents'][0]['address']['address_name']
        else :
            address = response.json()['documents'][0]['road_address']['address_name']
        return address if res !=0 and address is not None else res
    except:
        return 0
    
    
tmp['new_address']=tmp.apply(getAddress,axis=1)    
tmp.to_csv('tmp.csv',encoding='ANSI')  

# 결과를 df_store에다가 저장
for i,a in zip(tmp['id'],tmp['new_address']):
    df_store.loc[df_store['id']==i,'address']=a

nonDataList = list(df_store[((df_store['latitude']==0) & (df_store['address'].isnull()))]['id'])
df_store_af = df_store[~df_store['id'].isin(nonDataList)]

# 위도 경도가 Null인건 걍 넣어쥼...
df_store_af.loc[df_store_af['id']==220307,'latitude']=37.5302862
df_store_af.loc[df_store_af['id']==220307,'longitude']=126.9854131

df_store_af.to_csv('df_store_with_address.csv', encoding="ANSI", index=False)
```

In [6]:
df = pd.read_csv('df_store_with_address.csv', encoding="ANSI")

In [7]:
df.head()

,id,store_name,branch,area,tel,address,latitude,longitude,category,review_count,review_total_score
0,1,Agal,NaN,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,아구찜|포장마차,0,0
1,2,Assisy,NaN,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,카페,0,0
2,3,Battered Sole,NaN,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,피쉬앤칩스|펍,0,0
3,4,Chakyoung,NaN,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,레스토랑|카프레제,0,0
4,5,Delabobo,NaN,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,디저트카페|디저트,0,0


In [75]:
# 위도나 경도가 없는 것들
nonLonLat = df[((df['latitude']==0) | (df['longitude']==0))]
nonLonLat

,id,store_name,branch,area,tel,address,latitude,longitude,category,review_count,review_total_score
32023,32028,골뱅이연구소,NaN,발산역,010-7791-2995,서울 강서구 마곡동 794-1,0.0,0.0,NaN,1,5
37404,37409,교화,NaN,시흥,031-496-0805,경기도 시흥시 정왕동 2473 114호,0.0,0.0,NaN,0,0
45217,45222,그릴데미그라스,종로점,삼청동,02-723-1233,서울특별시 종로구 예장동 2-19,0.0,0.0,경양식|함박스테이크,1,3
67825,67832,녘,NaN,을지로3가,070-4150-0504,서울시 중구 수표동 65 1층,0.0,0.0,NaN,3,12
199022,199038,빈트리200.25,불광점,불광동,02-382-0282,서울특별시 은평구 불광동 8-27,0.0,0.0,카페,0,0
206748,206764,산수모밀,NaN,광주,062-233-5525,광주광역시 동구 금남로2가 15,0.0,0.0,모밀|돈까스,0,0
213592,213608,새미나,NaN,마포,02-716-3332,서울특별시 마포구 염리동 148-28,0.0,0.0,돈까스|만두,1,4
232513,232530,송이가든,NaN,청송,054-874-0066,경북 청송군 부동면 하의리 751,0.0,0.0,송이버섯전골|닭도리탕,1,4
337633,337656,장군씨푸드,NaN,연안부두,010-8880-9367,인천광역시 중구 항동7가 65-15,0.0,0.0,NaN,1,4
349583,349606,제이제이로스터스,NaN,화성시,031-227-6001,경기도 화성시 봉당읍 동화리 599-1,0.0,0.0,NaN,0,0


In [77]:
headers = {
"Authorization":"KakaoAK 66447c8b7057c042f65ac177c53ea385"
}
def getLonLat(addr):
    print(addr)
    print("------------")
    params = (
        ('query', addr),
    )
    response = requests.get('https://dapi.kakao.com/v2/local/search/address.json', headers=headers, params=params)

    list_result  = response.json()['documents']
    x=0
    y=0
    for i in list_result:
        print(i['address']['address_name'])
        print(i['address']['x'])
        print(i['address']['y'])
        x=i['address']['x']
        y=i['address']['y']
    return str(x)+"/"+str(y)
nonLonLat['Results'] = nonLonLat['address'].apply(getLonLat)

서울 강서구 마곡동 794-1
------------
서울 강서구 마곡동 794-1
126.838304106686
37.5600926354663
경기도 시흥시 정왕동 2473 114호
------------
서울특별시 종로구 예장동 2-19
------------
서울시 중구 수표동 65 1층
------------
서울특별시 은평구 불광동 8-27
------------
광주광역시 동구 금남로2가 15
------------
광주 동구 금남로2가 15
126.918422468796
35.1487767497779
서울특별시 마포구 염리동 148-28
------------
서울 마포구 염리동 148-28
126.945843963166
37.5463533228141
경북 청송군 부동면 하의리 751
------------
경북 청송군 주왕산면 하의리 751
129.110611511361
36.3789738662123
인천광역시 중구 항동7가 65-15
------------
인천 중구 항동7가 65-15
126.597873544277
37.4505354480654
경기도 화성시 봉당읍 동화리 599-1
------------
경기도 안양시 만안구 안양동 714-71
------------
경기 안양시 만안구 안양동 714-71
126.916034987667
37.3944749117724
서울시 종로구 청진동 24 르메이에르 종로1타운 지하1층
------------
서울 노원구 상계동 670-2번지 9단지상가 114호
------------
서울 노원구 상계동 670-2
127.059395452216
37.6614574251567
서울특별시 강남구 삼성동 157-37
------------
서울 강남구 삼성동 157-37
127.057637263187
37.5078092477983


C:\Users\multicampus\anaconda3\envs\pjt\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [78]:
nonLonLat[nonLonLat['Results']=='0/0'] 
# x = longitude
# y = latitude
# x/ y    == longitude / latitude

,id,store_name,branch,area,tel,address,latitude,longitude,category,review_count,review_total_score,Results
37404,37409,교화,NaN,시흥,031-496-0805,경기도 시흥시 정왕동 2473 114호,0.0,0.0,NaN,0,0,0/0
45217,45222,그릴데미그라스,종로점,삼청동,02-723-1233,서울특별시 종로구 예장동 2-19,0.0,0.0,경양식|함박스테이크,1,3,0/0
67825,67832,녘,NaN,을지로3가,070-4150-0504,서울시 중구 수표동 65 1층,0.0,0.0,NaN,3,12,0/0
199022,199038,빈트리200.25,불광점,불광동,02-382-0282,서울특별시 은평구 불광동 8-27,0.0,0.0,카페,0,0,0/0
349583,349606,제이제이로스터스,NaN,화성시,031-227-6001,경기도 화성시 봉당읍 동화리 599-1,0.0,0.0,NaN,0,0,0/0
386303,386327,춘천숯불닭갈비,NaN,종로,02-732-1188,서울시 종로구 청진동 24 르메이에르 종로1타운 지하1층,0.0,0.0,숯불닭갈비|닭갈비,0,0,0/0


In [79]:
nonLonLat.loc[nonLonLat['id']==37409,'Results']=str(127.0577131)+"/"+str(37.5076276)
nonLonLat.loc[nonLonLat['id']==45222,'Results']=str(126.9889093)+"/"+str(37.5600152)
nonLonLat.loc[nonLonLat['id']==67832,'Results']=str(126.9868719)+"/"+str(37.5676577)
nonLonLat.loc[nonLonLat['id']==199038,'Results']=str(126.9327362)+"/"+str(37.6136145)
nonLonLat.loc[nonLonLat['id']==349606,'Results']=str(126.9563991)+"/"+str(37.2171489)
nonLonLat.loc[nonLonLat['id']==386327,'Results']=str(126.9776657)+"/"+str(37.5708033)


In [80]:
nonLonLat['longitude'] = nonLonLat['Results'].str.split('/').str[0]
nonLonLat['latitude'] = nonLonLat['Results'].str.split('/').str[1]

C:\Users\multicampus\anaconda3\envs\pjt\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\multicampus\anaconda3\envs\pjt\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [81]:
nonLonLat

,id,store_name,branch,area,tel,address,latitude,longitude,category,review_count,review_total_score,Results
32023,32028,골뱅이연구소,NaN,발산역,010-7791-2995,서울 강서구 마곡동 794-1,37.5600926354663,126.838304106686,NaN,1,5,126.838304106686/37.5600926354663
37404,37409,교화,NaN,시흥,031-496-0805,경기도 시흥시 정왕동 2473 114호,37.5076276,127.0577131,NaN,0,0,127.0577131/37.5076276
45217,45222,그릴데미그라스,종로점,삼청동,02-723-1233,서울특별시 종로구 예장동 2-19,37.5600152,126.9889093,경양식|함박스테이크,1,3,126.9889093/37.5600152
67825,67832,녘,NaN,을지로3가,070-4150-0504,서울시 중구 수표동 65 1층,37.5676577,126.9868719,NaN,3,12,126.9868719/37.5676577
199022,199038,빈트리200.25,불광점,불광동,02-382-0282,서울특별시 은평구 불광동 8-27,37.6136145,126.9327362,카페,0,0,126.9327362/37.6136145
206748,206764,산수모밀,NaN,광주,062-233-5525,광주광역시 동구 금남로2가 15,35.1487767497779,126.918422468796,모밀|돈까스,0,0,126.918422468796/35.1487767497779
213592,213608,새미나,NaN,마포,02-716-3332,서울특별시 마포구 염리동 148-28,37.5463533228141,126.945843963166,돈까스|만두,1,4,126.945843963166/37.5463533228141
232513,232530,송이가든,NaN,청송,054-874-0066,경북 청송군 부동면 하의리 751,36.3789738662123,129.110611511361,송이버섯전골|닭도리탕,1,4,129.110611511361/36.3789738662123
337633,337656,장군씨푸드,NaN,연안부두,010-8880-9367,인천광역시 중구 항동7가 65-15,37.4505354480654,126.597873544277,NaN,1,4,126.597873544277/37.4505354480654
349583,349606,제이제이로스터스,NaN,화성시,031-227-6001,경기도 화성시 봉당읍 동화리 599-1,37.2171489,126.9563991,NaN,0,0,126.9563991/37.2171489


In [94]:
for i in nonLonLat['id']:
    print(i)
    df.loc[df['id']==i,'latitude']=nonLonLat[nonLonLat['id']==i]['latitude']
    df.loc[df['id']==i,'longitude']=nonLonLat[nonLonLat['id']==i]['longitude']

32028
37409
45222
67832
199038
206764
213608
232530
337656
349606
359500
386327
396326
416934


In [96]:
# 위도나 경도가 없는 것들 확인
check = df[((df['latitude']==0) | (df['longitude']==0))]
check

,id,store_name,branch,area,tel,address,latitude,longitude,category,review_count,review_total_score


In [99]:
df.isnull().sum()

id                         0
store_name                 0
branch                444217
area                   16730
tel                    10860
address                   45
latitude                   0
longitude                  0
category              183439
review_count               0
review_total_score         0
dtype: int64

In [107]:
address_null =df[df['address'].isnull()]

## 아.. 주소가 null인거 포함안함...환장

In [108]:

# 카카오 API 써서 주소가 덜 작성된거는 위도 경도로 새 주소로 업뎃해쥼
headers = {
"Authorization":"KakaoAK 66447c8b7057c042f65ac177c53ea385"
}
def getAddress(row):
    
    params = (
        ('x', row['longitude']),
        ('y', row['latitude']),
        ('input_coord', 'WGS84'),
    )
    print(row['id'])

    response = requests.get('https://dapi.kakao.com/v2/local/geo/coord2address.json', headers=headers, params=params)
    res = response.json()['meta']['total_count']
    address = None
    try:
        if response.json()['documents'][0]['road_address'] is None:
            address =response.json()['documents'][0]['address']['address_name']
        else :
            address = response.json()['documents'][0]['road_address']['address_name']
        return address if res !=0 and address is not None else None
    except:
        return None
    
    
address_null['address']=address_null.apply(getAddress,axis=1) 

33767
34376
35769
40602
41849
52082
74421
96352
100035
100036
100037
100038
109414
111914
116610
122279
125776
131890
132979
154298
184675
217257
218448
294225
294226
319093
327578
338761
338762
339083
342648
348835
355543
356285
368155
368156
370904
370905
379936
387459
419085
434936
444536
446130
449577


C:\Users\multicampus\anaconda3\envs\pjt\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [109]:
address_null

,id,store_name,branch,area,tel,address,latitude,longitude,category,review_count,review_total_score
33762,33767,공원국수,NaN,NaN,054-776-4532,경상북도 경주시 동천로42번길 16,35.8515,129.228,NaN,0,0
34371,34376,공포의생삼겹살,NaN,NaN,033-535-7177,강원도 동해시 한섬로 113,37.5215,129.116,NaN,0,0
35764,35769,광치아줌마,NaN,NaN,033-482-5500,강원도 양구군 양구읍 관공서로 21-7,38.1079,127.99,NaN,0,0
40597,40602,국수맨,NaN,반월당,053-431-0777,대구 중구 남성로 69,35.8675,128.591,국수|석쇠불고기,0,0
41844,41849,굼터,NaN,NaN,033-581-5050,강원도 태백시 장성로 230,37.1011,129.021,NaN,0,0
52077,52082,김밥천국,NaN,NaN,054-474-3360,경상북도 구미시 인동31길 22-17,36.0964,128.434,NaN,0,0
74414,74421,다래순대,NaN,NaN,033-335-0589,강원도 평창군 진부면 청송로 90-6,37.6364,128.556,NaN,0,0
96344,96352,도원반점,NaN,NaN,054-635-8286,경상북도 영주시 중앙로80번길 10,36.8225,128.625,NaN,0,0
100027,100035,돈통령,NaN,NaN,NaN,서울특별시 성동구 왕십리로14길 32-3,37.5485,127.047,NaN,0,0
100028,100036,돈통령,NaN,NaN,NaN,서울특별시 성동구 왕십리로14길 32-3,37.5485,127.047,NaN,0,0


In [110]:
for i in address_null['id']:
    print(i)
    df.loc[df['id']==i,'address']=address_null[address_null['id']==i]['address']
df

33767
34376
35769
40602
41849
52082
74421
96352
100035
100036
100037
100038
109414
111914
116610
122279
125776
131890
132979
154298
184675
217257
218448
294225
294226
319093
327578
338761
338762
339083
342648
348835
355543
356285
368155
368156
370904
370905
379936
387459
419085
434936
444536
446130
449577


,id,store_name,branch,area,tel,address,latitude,longitude,category,review_count,review_total_score
0,1,Agal,NaN,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.5569,126.927,아구찜|포장마차,0,0
1,2,Assisy,NaN,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.1507,126.89,카페,0,0
2,3,Battered Sole,NaN,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535,126.992,피쉬앤칩스|펍,0,0
3,4,Chakyoung,NaN,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.1586,129.175,레스토랑|카프레제,0,0
4,5,Delabobo,NaN,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.5599,126.841,디저트카페|디저트,0,0
...,...,...,...,...,...,...,...,...,...,...,...
459971,459996,행복부대찌개,NaN,녹양동,031-877-2410,경기도 의정부시 녹양동 154-6,37.7579,127.033,NaN,0,0
459972,459997,행복부페,NaN,안중읍,031-682-1733,경기도 평택시 안중읍 용성리 718-3,37.0027,126.945,NaN,0,0
459973,459998,행복분식,NaN,퇴촌면,031-8071-2668,경기도 광주 퇴촌면 도수리 611-6,37.4717,127.316,NaN,0,0
459974,459999,행복분식,NaN,소주동,055-312-3713,경상남도 양산시 소주동 147-1,35.4079,129.159,NaN,0,0


In [111]:
df.isnull().sum()

id                         0
store_name                 0
branch                444217
area                   16730
tel                    10860
address                    0
latitude                   0
longitude                  0
category              183439
review_count               0
review_total_score         0
dtype: int64

In [112]:
# 드디어 최종 df 끝...이라고 믿게 해주세요 ^^..
df.to_csv('final_store.csv',encoding="ANSI")

In [113]:
con = sqlite3.connect("db.sqlite3")
cur = con.cursor()
review = pd.read_sql_query("SELECT * from api_review", con)
con.close()

In [119]:
review_list = pd.DataFrame(review['store_id'].unique(),columns=["store_id"])

In [120]:
review_list

,store_id
0,15
1,18
2,19
3,37
4,38
...,...
45695,360493
45696,360499
45697,360505
45698,360514


In [121]:
test = pd.merge(review_list, df, left_on='store_id', right_on = 'id')

In [123]:
test.isnull().sum()

store_id                  0
id                        0
store_name                0
branch                40907
area                    276
tel                    1282
address                   0
latitude                  0
longitude                 0
category               1613
review_count              0
review_total_score        0
dtype: int64

In [ ]:
# 다행이다.. 없는듯..

In [143]:
import os
import shutil

DATA_DIR = "../../data"
DUMP_FILE = os.path.join(DATA_DIR, "dump_store.pkl")
pd.to_pickle(df, DUMP_FILE)

In [127]:
data_store = pd.read_pickle(os.path.join(DATA_DIR, "dump_store.pkl"))
print(data_store.head())

   id     store_name branch  area            tel                address  \
0   1           Agal    NaN    홍대  010-6689-5886   서울특별시 마포구 동교동 170-13   
1   2         Assisy    NaN    광주   062-367-0700    광주광역시 서구 농성동 631-33   
2   3  Battered Sole    NaN   이태원    02-749-6867   서울특별시 용산구 이태원동 118-9   
3   4      Chakyoung    NaN  달맞이길   051-756-5566  부산광역시 해운대구 중2동 1509-5   
4   5       Delabobo    NaN   발산역   02-2667-9854      서울특별시 강서구 등촌동 689   

  latitude longitude   category  review_count  review_total_score  
0  37.5569   126.927   아구찜|포장마차             0                   0  
1  35.1507    126.89         카페             0                   0  
2   37.535   126.992    피쉬앤칩스|펍             0                   0  
3  35.1586   129.175  레스토랑|카프레제             0                   0  
4  37.5599   126.841  디저트카페|디저트             0                   0  
